# Genomic DNA per-base coverage calculation

### This notebook accompanies the paper "Illuminating Genetic Mysteries of the Dead Sea Scrolls"
#### Author: Moran Neuhof

The following notebook describes the code behind the removal of contamination in BAM files.
It follows the method described in the "Genomic DNA coverage calculation" section of the STAR Methods.

In [ ]:
import os
from Bio import SeqIO
import pysam
from collections import Counter
import gzip

folder_join = os.path.join  # aliasing

The code below calculates the per-base coverage of the reference genome.
It only considers the special case of Alelle Frequency = 1 (no heterozygosity, no contamination) to avoid bias.
This strict condition is used in our downstream analysis (i.e. clustering, pairwise similarity, etc.)

In [ ]:
def calc_af_per_base(ref, val_list):
    """
    Receive reference nucleotide and value list at this position.
    Return the reference nucleotide (uppercase), ALT, and the AF
    """
    ref = ref.upper()  # upper case all ref
    counter = Counter(val_list)  # counting the values in val_list
    if len(counter) == 1:  
        if counter[ref] == 0:  # we only have ALT
            af = 1
            for alt in counter.keys():
                return ref, alt, af  # we have af=1 and alt
        else:  # we only have REF
            af = 0
            return ref, ref, af
    elif len(counter) == 2:  # ref/alt situation
        # we have mixed SNP! print the counter and move on.
        print(f"We have alt/ref mix here, ignoring: {counter}")
        return None, None, None
    else:  # we have more than 2 alelles... 
        print(f"We have more than 2 alelles, ignoring: {counter}")
        return None, None, None

### User parameters:

In [ ]:
MIN_COVERAGE = 3  # Decide on your coverage
bam_fname = None  # choose BAM file name to analyze

out_file_name = f'{bam_fname}.cov{min_coverage}.af_alt_cov.tsv.gz'
FASTA_DB = "mammal_genomes/sheep/Ovis_aries/GCF_000298735.2_Oar_v4.0_genomic.fna"  # sheep fasta reference genome
record_dict = SeqIO.to_dict(SeqIO.parse(FASTA_DB, "fasta"))  # load genome

Run and get coverage:

In [ ]:
with gzip.open(out_file_name, 'wt') as outfile:  # writing a compressed file
    # read the bam file
    print("\t".join(['CHR', 'POS', 'REF', 'ALT', 'AF', 'COV']), file=outfile)  # printing header to outfile
    with pysam.AlignmentFile(bam_fname, "rb") as bamfile:
        for pileupcolumn in bamfile.pileup():  # for each position
            cov = pileupcolumn.n  # get coverage at position
            if cov < MIN_COVERAGE:  # minimum coverage
                continue  # skip low coverage
            
            pos = pileupcolumn.pos  # current position
            ref_chr = pileupcolumn.reference_name  # reference/chromosome name/ID
            ref_nuc = record_dict[ref_chr][pos]  # the nucleotide in the reference genome
            
            try:  # find the value of this specific position in all covering reads
                all_nucs_in_pos = [p.alignment.query[pos - p.alignment.reference_start]  for p in pileupcolumn.pileups]
            except IndexError:
                print("problem with position:", ref_chr, pos + 1, ref_nuc, cov) 
            
            # getting REF, ALT, AF
            REF, ALT, AF = calc_af_per_base(ref_nuc, all_nucs_in_pos)
            
            if REF:  # if the result was not ambiguous
                print("\t".join([ref_chr, str(pos+1), REF, ALT, str(AF), str(cov)]), file=outfile)  # pos will be printed as 1-based

print("Done.")
